In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.inspection import permutation_importance

In [ ]:
raw_df = pd.read_csv('stats.csv')

In [ ]:
df = raw_df.copy()
df.drop('last_name, first_name',axis = 1,inplace=True)
df.drop('player_id',axis = 1,inplace=True)

CREATING A CLASSIFICATION PROBLEM 
BINNING OBPS 
An OPS scale

According to https://en.wikipedia.org/wiki/On-base_plus_slugging
"Bill James, in his essay titled "The 96 Families of Hitters"[4] uses seven different categories for classification by OPS:

| Category | Classification	| OPS range       |
|----------|----------------|-----------------|
|A         |	Great	    |.9000 and higher |
|B         |	Very good	|.8334 to .8999   |
|C         |Above average   |.7667 to .8333   |
|D         |	Average	    | .7000 to .7666  |
|E         |Below average	|.6334 to .6999   |
|F         |	Poor	    |.5667 to .6333   |
|G         |	Very poor   |.5666 and lower  |




I have created a scale such that we are classifying below, average and above average hitters based on their ops
The categories for classification by OPS I have created: 


| Category | Classification	| OPS range       |
|----------|----------------|-----------------|
|   1      |Above average   |.7667 and higher |
|0         |	Average	    | .7000 to .7666  |
|-1        |Below average	|.6999  and lower |


In [ ]:
#new dataset for classification
c_df = df.copy()

# Define the OPS ranges and corresponding classifications
ops_ranges = {
    1: (0.7667, float('inf')),
    0: (0.7000, 0.7666),
    -1: (-float('inf'), 0.6999)
}

# Create an empty list to store the classifications
classifications = []

# Iterate over each OPS value in your dataset
for ops_value in c_df['on_base_plus_slg']:
    # Iterate over each classification and its corresponding OPS range
    for classification, (lower_bound, upper_bound) in ops_ranges.items():
        # Check if the OPS value falls within the current classification's range
        if lower_bound <= ops_value < upper_bound:
            # If it does, append the classification to the list
            classifications.append(classification)
            # Break out of the inner loop since we found the correct classification
            break
    else:
        # If the OPS value didn't fall within any range, append a default value
        classifications.append('Unknown')

# Add the classifications to your dataset as a new column
c_df['Classification'] = classifications



#double checking that there are no missing or duplicated values for saftey 

if c_df.isna().any().any():
    print("There are missing values in the classification dataframe c_df.")
else:
    print("There are no missing values in the classification dataframe c_df")

if c_df.duplicated().any().any():
    print("There are duplicated values in the classification dataframe c_df.")
else:
    print("There are no duplicated values in the classification dataframe c_df")


In [ ]:
#train test split datasets

c_train_df = c_df[(c_df['year'] == 2021) | (c_df['year'] == 2020)]
c_val_df = c_df[c_df.year==2022]
c_test_df = c_df[c_df.year==2023]